In [11]:
import os, platform, sys, time
from datetime import date
import cv2
import numpy as np
from collections import defaultdict
import face_recognition
import imutils
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Activation, AveragePooling2D
from PIL import Image
import matplotlib.pyplot as plt

In [12]:
IMG_SIZE = 24

In [13]:
def predict(img, model):
    img = np.array(Image.fromarray(img, 'RGB').convert('L'))
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).astype('float32')
    img /= 255
    img = img.reshape(1, IMG_SIZE, IMG_SIZE, 1)
    prediction = model.predict(img)
    print(prediction)
    if prediction < 0.1:
        prediction = True
    else:
        prediction = False
    # print("closed : ", prediction, " ", end='')
    return prediction

In [14]:
def init():
    # print("init function")
    face_cascPath = 'haarcascade_frontalface_alt.xml'
    # open_eye_cascPath = "haarcascade_eye.xml"
    # open_eyeglasses_cascPath = 'haarcascade_eye_tree_eyeglasses.xml'
    left_eye_cascPath = 'haarcascade_lefteye_2splits.xml'
    right_eye_cascPath = 'haarcascade_righteye_2splits.xml'

    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + face_cascPath)
    """ open_eyes_detector = cv2.CascadeClassifier(
        cv2.data.haarcascades+open_eye_cascPath)
    open_eyesglasses_detector = cv2.CascadeClassifier(
        cv2.data.haarcascades+open_eyeglasses_cascPath) """
    left_eye_detector = cv2.CascadeClassifier(
        cv2.data.haarcascades + left_eye_cascPath)
    right_eye_detector = cv2.CascadeClassifier(
        cv2.data.haarcascades + right_eye_cascPath)

    model = Sequential([Conv2D(6, (3, 3), input_shape=(24, 24, 1)),
                        Activation("relu"),
                        AveragePooling2D(),
                        Conv2D(16, (3, 3), input_shape=(24, 24, 1)),
                        Activation("relu"),
                        AveragePooling2D(),
                        Flatten(),
                        Dense(120),
                        Activation('relu'),
                        Dense(84),
                        Activation('relu'),
                        Dense(1),
                        Activation('sigmoid')
                        ])
    model.load_weights("eye_status.h5")

    # print("[LOG] Collecting images ...")
    images = []
    for direc, _, files in os.walk("ImagesTrain"):
        for file in files:
            if file.endswith("jpeg") or file.endswith("jpg"):
                images.append(os.path.join(direc, file))
    return model, face_detector, left_eye_detector, right_eye_detector, images

In [15]:
def isBlinking(history):
    print("history" , history)
    """ for i in range(maxFrames):
        pattern = '1' + '0'*(i+1) + '1'
        if pattern in history:
            return True
    return False """
    if '1' in history and '0' in history:
        return True
    else:
        return False

def detect_and_display(model, frame, face_detector, left_eye_detector, right_eye_detector, data, eyes_detected):
    # resize the frame
    frame = cv2.resize(frame, (0, 0), fx=0.6, fy=0.6)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    faces = face_detector.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=5,
        minSize=(50, 50),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    if len(faces) > 0:
        print("face detected")
        (x, y, w, h) = faces[0]

        # separate the face into left and right sides
        left_face = frame[y:y + h, x + int(w / 2):x + w]
        left_face_gray = gray[y:y + h, x + int(w / 2):x + w]

        right_face = frame[y:y + h, x:x + int(w / 2)]
        right_face_gray = gray[y:y + h, x:x + int(w / 2)]

        # Detect the left eye
        left_eye = left_eye_detector.detectMultiScale(
            left_face_gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # Detect the right eye
        right_eye = right_eye_detector.detectMultiScale(
            right_face_gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        eye_status = '1'
        closed = False  # we suppose the eyes are closed

        # For each eye check wether the eye is closed.
        # If one is open we conclude the eyes are open.
        for (ex, ey, ew, eh) in right_eye:
            pred = predict(right_face[ey:ey + eh, ex:ex + ew], model)
            closed = closed or pred
        for (ex, ey, ew, eh) in left_eye:
            pred = predict(left_face[ey:ey + eh, ex:ex + ew], model)
            closed = closed or pred
        if closed:
            eye_status = '0'
        eyes_detected += eye_status
        # print("eyes_detected" , eyes_detected)
        if isBlinking(eyes_detected):
            # face recognition
            encoding = face_recognition.face_encodings(rgb, [(y, x + w, y + h, x)])[0]
            matches = face_recognition.compare_faces(data["encodings"], encoding)
            diff = face_recognition.face_distance(data['encodings'], encoding)
            idx = np.argmin(diff)
            if matches[idx]:
                name = data['names'][idx]
                return name, eyes_detected
    return False, eyes_detected


In [16]:
def process_and_encode(images):
    # initialize the list of known encodings and known names
    # start = datetime.datetime.now()
    known_encodings = []
    known_names = []
    # print("[LOG] Encoding faces ...")

    for image_path in images:
        # Load image
        image = cv2.imread(image_path)
        # Convert it from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # detect face in the image and get its location (square boxes coordinates)
        boxes = face_recognition.face_locations(image, model='hog')

        # Encode the face into a 128-d embeddings vector
        encoding = face_recognition.face_encodings(image, boxes)

        # the person's name is the name of the folder where the image comes from
        name = image_path.split(os.path.sep)[-1]

        if len(encoding) > 0:
            known_encodings.append(encoding[0])
            #print(type(encoding[0]))
            known_names.append(name)
    # print(datetime.datetime.now() - start)
    return {"encodings": known_encodings, "names": known_names}

(model, face_detector,left_eye_detector, right_eye_detector, images) = init()
data = process_and_encode(images)
#print(data['encodings'])

In [17]:
#print(data)

{'encodings': [array([-1.44782379e-01,  6.87690973e-02,  1.73213691e-01,  2.50547081e-02,
       -2.01581761e-01, -3.14304046e-02, -1.42977014e-03, -9.54940021e-02,
        1.21585026e-01, -1.30392358e-01,  1.04645982e-01, -1.13912910e-01,
       -2.19910994e-01,  1.83378518e-01, -3.86749916e-02,  2.02095062e-01,
       -1.22953802e-01, -1.55431092e-01, -1.67476639e-01, -5.42518906e-02,
       -1.08347498e-02,  8.59467089e-02, -4.50081229e-02,  1.04137525e-01,
       -1.60984516e-01, -3.04977030e-01, -4.40315381e-02, -5.22042438e-02,
        4.57234979e-02,  1.95716210e-02, -5.29527739e-02,  7.99582824e-02,
       -1.61494493e-01, -2.50106379e-02,  5.90142831e-02,  1.13795899e-01,
       -7.81761259e-02, -7.38957822e-02,  2.59470612e-01, -1.96652412e-02,
       -2.92149514e-01,  2.80841105e-02,  1.31303862e-01,  2.37295642e-01,
        1.66224763e-01,  4.71508913e-02,  6.72675371e-02, -7.39109218e-02,
        1.11176625e-01, -3.82212251e-01,  2.32122019e-02,  1.19901523e-01,
        9.

In [1]:
cap = cv2.VideoCapture(0)
eyes_detected = ""
while True:
    ret, frame = cap.read()
    if ret:
        name, eyes_detect = detect_and_display(model, frame, face_detector, left_eye_detector, right_eye_detector,
                                               data, eyes_detected)
        eyes_detected = eyes_detect
        plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
        plt.show()
        if name:
            # print(datetime.datetime.now()-start)
            print(name)
            break
    else:
        break

cap.release()

NameError: name 'cv2' is not defined